In [1]:
import ast
from collections import Counter
from gensim import utils, matutils 
from gensim.corpora.dictionary import Dictionary
from gensim.utils import deprecated
import itertools
from nltk.corpus import stopwords
import numpy as np
from numpy import linalg as LA
from numpy.random import binomial
from numpy import dot, float32 as REAL, memmap as np_memmap, \
    double, array, zeros, vstack, sqrt, newaxis, integer, \
    ndarray, sum as np_sum, prod, argmax
from numpy.linalg import norm
import math, copy, time
import warnings 

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context(context="talk")
%matplotlib inline

from keras import backend as K
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras import backend as K
import keras
from keras.utils import to_categorical
from keras.preprocessing import sequence
from keras.models import load_model

import os, sys, re, io, nltk, torch
import pandas as pd

from scipy import stats
from scipy.stats import bernoulli
from scipy.spatial import distance
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils import class_weight
from sklearn.preprocessing import Normalizer, normalize
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix, recall_score, precision_score

from six import string_types, integer_types
from six.moves import zip, range
if not sys.warnoptions:
    warnings.simplefilter("ignore")

from tqdm import tqdm, trange
import time
import transformers
from transformers import (
    WEIGHTS_NAME,
    AdamW,
    BertConfig,
    BertForMaskedLM,
    BertForSequenceClassification,
    BertTokenizer,
    CamembertConfig,
    CamembertForMaskedLM,
    CamembertTokenizer,
    DistilBertConfig,
    DistilBertForMaskedLM,
    DistilBertTokenizer,
    GPT2Config,
    GPT2LMHeadModel,
    GPT2Tokenizer,
    OpenAIGPTConfig,
    OpenAIGPTLMHeadModel,
    OpenAIGPTTokenizer,
    RobertaConfig,
    RobertaForMaskedLM,
    RobertaTokenizer,
    get_linear_schedule_with_warmup,
)

import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.autograd import Variable
import re
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import *

stemmer_sn = SnowballStemmer("english")
stemmer = PorterStemmer()

stoplist = stopwords.words("english")
lemmatizer=WordNetLemmatizer()


'''
Compatible with tensorflow backend
gamma entre más alto más tolerante (queremos un gamma chico pero no demasiado! 0.25 - 5)
alpha pesos por clase (weights)
'''
def focal_loss(gamma=2., weights=1):   #weights np.asarray()
    weights= K.variable(weights)
    def focal_loss_fixed(y_true, y_pred):
        y_true = K.clip(y_true, K.epsilon(),1)
        y_pred = K.clip(y_pred,K.epsilon(),1)
        return - K.sum(weights* K.pow(1. - y_pred, gamma)* y_true * K.log(y_pred), axis=-1) 
    return focal_loss_fixed

keras.losses.focal_loss=keras.losses.MSE
keras.losses.focal_loss_fixed=keras.losses.MSE

n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

if torch.cuda.is_available():     
    device = torch.device("cuda")
    print('There are',torch.cuda.device_count(),'GPU(s) available.')
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
class Dummy_Embeddings(nn.Module):
    def __init__(self, d_model, dummy_vectors): 
        super(Dummy_Embeddings, self).__init__()
        aux = torch.from_numpy(dummy_vectors)#, dtype='float32'))
        self.index2dummy = nn.Embedding(aux.size()[0], d_model)
        self.index2dummy.weigth=nn.Parameter(aux)
        self.index2dummy.weigth.requires_grad=False
        self.d_model = d_model
        
    def forward(self, x):
        aux=x.numpy()
        #print ("aux original", aux)
        #print ("aux restado...",aux-np.ones(aux.shape))
        new_x= aux #- np.ones(aux.shape)
        new_x= torch.from_numpy(new_x)
        return self.index2dummy(new_x.long()) * math.sqrt(self.d_model) #debiese retornar matriz de batch_size x [ind_tw, k1,k2,k3,k4,k5,k6] (si son 6 modelos)
    
def match(objetos,ejemplo):
    i=0
    for obj in objetos:
        if obj==ejemplo:
            return i
        i+=1
        
def plot_confusion_matrix(cm, target_names, title='Confusion matrix (f1-score)',cmap=None, normalize=True):    
    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy
    if cmap is None:
        cmap = plt.get_cmap('Blues')
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap, vmin=0, vmax=1)
    plt.title(title)
    plt.colorbar()
    
    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)
    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()
    
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

def cleaner(post): 
    #re.sub(r'([a-z])\1+', r'\1', "user i think that ' s all you loooooove ")
    s= re.sub(r"http\S+", " html ", post)   ##########elimina http    
    s= re.sub(r'#\w+ ?', " hashtag ", s) ########3
    s= re.sub(r'@\w+ ?', " user ", s)##############    
    s= s.lower()
    s=emoji_pattern.sub(r'', s)
    s=re.sub(r"'\b", " ' ", s)
    s=re.sub(r"\b'\b", " ' ", s)
    s=re.sub(r"\b’", " ’ ", s)
    s=re.sub(r"‘\b", " ‘ ", s)
    s=re.sub(r"\b’\b", " ‘ ", s)
    s = re.sub(r"-", " - ", s)
    s = re.sub(r"\(", " ", s)
    s = re.sub(r"\)", " ", s)
    s = re.sub(r"\?", " ? ", s)    
    s = re.sub(r"\/", " ", s)
    s = re.sub(r"' ", " ' ", s)
    s = re.sub(r" '", " ' ", s)
    s = re.sub(r"\!", " ! ", s)
    s=re.sub("[\.]+", " . ", s)
    s=re.sub("[\,]+", " , ", s)
    s=re.sub("[\;]+", " ; ", s)
    s=re.sub("[\:]+", " : ", s)
    s=re.sub('[\"]+', ' " ', s)
    s=re.sub(r'\b[0-9]\b', " number ",  s)
    s=re.sub(r'\b[0-9]*[0-9]\b', " number ",  s)    
    s=re.sub(r'\b”', ' " ', s)
    sl= list(s.split())
    #sl= [stemmer.stem(wd) for wd in sl]
    ############3
    sl_2=[]
    for wd in sl:
        try: 
            q=Word2Index[wd]
            sl_2.append(wd)
        except:
            try: 
                if stemmer.stem(wd) in Word2Index.keys():
                    sl_2.append(stemmer.stem(wd))
                    #print ("dejamos el stemmed de", wd)
                elif lemmatizer.lemmatize(wd) in Word2Index.keys():
                    sl_2.append(lemmatizer.lemmatize(wd))
                else:
                    sl_2.append(wd)
                    #print ("dejamos el lema de", wd)
                #print ("veamos si ",stemmer.stem(wd), "\tdel termino ", wd, "\testa:")
                #r=Word2Index[stemmer.stem(wd)]
                #sl_2.append(stemmer.stem(wd))
                
            except:
                sl_2.append(wd)
    sl=sl_2
    ##########
    s=' '.join([word for word in sl])# if word not in stoplist])
    return s, sl


Using TensorFlow backend.
/home/casapanshop/anaconda2/envs/newpy3_tf1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/casapanshop/anaconda2/envs/newpy3_tf1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/casapanshop/anaconda2/envs/newpy3_tf1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1060 6GB


In [2]:
def predict_data(trained, x_train, x_val, x_test, etiq, etiq_v, etiq_t, name_model):
    etiq = etiq.astype("int")
    trainPredict = trained.predict(x_train, batch_size=32)
    trainPredict=[np.argmax(pred) for pred in trainPredict]
    acc= accuracy_score(etiq, trainPredict)  
    f1=f1_score(etiq, trainPredict, average=None)  # labels=np.unique(trainPredict)
    f1_ma=f1_score(etiq, trainPredict, average='macro')  # labels=np.unique(trainPredict)
    #matriz=normalize(confusion_matrix(etiq, trainPredict))
    print ("")
    print ("Accuracy sobre Train", name_model, ":",acc)  
    print ("F1-score None sobre Train", name_model, ":",f1)
    print ("F1-score macro sobre Train", name_model, ":",f1_ma)
    ########################################
    trainPredict = trained.predict(x_val, batch_size=32)
    trainPredict=[np.argmax(pred) for pred in trainPredict]
    acc= accuracy_score(etiq_v, trainPredict)  
    f1=f1_score(etiq_v, trainPredict, average=None)  # labels=np.unique(trainPredict)
    f1_ma=f1_score(etiq_v, trainPredict, average='macro')  # labels=np.unique(trainPredict)
    #matriz=normalize(confusion_matrix(etiq_v, trainPredict))
    print ("")
    print ("Accuracy sobre Val", name_model, ":",acc)  
    print ("F1-score None sobre Val", name_model, ":",f1)
    print ("F1-score macro sobre Val", name_model, ":",f1_ma)
    ########################################
    trainPredict = trained.predict(x_test, batch_size=32)
    trainPredict=[np.argmax(pred) for pred in trainPredict]
    acc_t= accuracy_score(etiq_t, trainPredict)  
    f1_t=f1_score(etiq_t, trainPredict, average=None)  # labels=np.unique(trainPredict)
    f1_ma_t=f1_score(etiq_t, trainPredict, average='macro')  # labels=np.unique(trainPredict)
    matriz_t=normalize(confusion_matrix(etiq_t, trainPredict))
    print ("")
    print ("Accuracy sobre Test", name_model, ":",acc_t)  
    print ("F1-score None sobre Test", name_model, ":",f1_t)
    print ("F1-score macro sobre Test", name_model, ":",f1_ma_t)
    
    return f1_ma_t, f1_t, acc_t, matriz_t
    

In [3]:
tokenizer_loaded = BertTokenizer.from_pretrained('bert-base-uncased')
model_loaded = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4)#'bert-base-uncased',  num_labels=4) 
idx_2_token = tokenizer_loaded.ids_to_tokens
Word2Index=tokenizer_loaded.vocab
M_BERT_space = model_loaded.bert.embeddings.word_embeddings.weight.detach().cpu().numpy()
transformer = Normalizer().fit(M_BERT_space) 
M_BERT_space=transformer.transform(M_BERT_space)

val=pd.read_csv("../../Fine-Tuning/CSV_Harassment/val_format.csv")
test=pd.read_csv("../../Fine-Tuning/CSV_Harassment/test_format.csv")

MAX_LEN = 50

print ("Conteo por stance val\n", Counter(val['Label']))
print ("Conteo por stance test\n", Counter(test['Label']))


Conteo por stance val
 Counter({'NonH': 1493, 'SexualH': 525, 'IndirectH': 71, 'PhysicalH': 36})
Conteo por stance test
 Counter({'NonH': 1601, 'SexualH': 340, 'IndirectH': 106, 'PhysicalH': 76})


In [4]:
dict_cl=dict()
dict_cl[0]="NonH"
dict_cl[1]="IndirectH"
dict_cl[2]="PhysicalH"
dict_cl[3]="SexualH"

#sdqc
d_lab=dict()
d_lab["NonH"]=0
d_lab["IndirectH"]=1
d_lab["PhysicalH"]=2
d_lab["SexualH"]=3

etiq_v=[d_lab[y] for y in val['Label']]
etiq_t=[d_lab[y] for y in test['Label']]


n_labels_val = np.array(etiq_v)
y_val=to_categorical(n_labels_val,num_classes=4)
sentences_val = val['Tweet'].values
input_ids_val = []
i=0
for sent in sentences_val:
    temp=cleaner(sent)
    encoded_sent2 = tokenizer_loaded.encode(temp[0],add_special_tokens = False) 
    input_ids_val.append(encoded_sent2)    
input_ids_val = pad_sequences(input_ids_val, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")


n_labels_test = np.array(etiq_t)
y_test=to_categorical(n_labels_test,num_classes=4)
sentences_test = test['Tweet'].values
input_ids_test = []
i=0
for sent in sentences_test:
    temp=cleaner(sent)
    encoded_sent2 = tokenizer_loaded.encode(temp[0],add_special_tokens = False)           
    input_ids_test.append(encoded_sent2)
input_ids_test = pad_sequences(input_ids_test, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")


shape_val=np.asarray(input_ids_val).shape
x_val=np.zeros((shape_val[0], shape_val[1], 768))
i=0
for in_id in input_ids_val:
    x_val[i]=M_BERT_space[in_id]
    i+=1    
shape_test=np.asarray(input_ids_test).shape
x_test=np.zeros((shape_test[0], shape_test[1], 768))
i=0
for in_id in input_ids_test:
    x_test[i]=M_BERT_space[in_id]
    i+=1

x_test.shape


(2123, 50, 768)

In [5]:
def predecir_modelos(bs,predichos_all):  #best fit
    temp=predichos_all.copy()
    final=[np.argmax(pred) for pred in temp]  
    confianzas=[temp[i][final[i]] for i in range(len(final))]
    predichos_new=[]
    for i in range(len(temp)):
        indice=final[i]%4
        predichos_new.append(int(indice))
    return predichos_new,final,confianzas    

def predecir_modelos_norm(bs,predichos_all):  #normalizado
    temp=predichos_all.copy()
    final=[]
    confianzas=[]
    for pred in temp:   
        aux=np.ones(4)
        for i in range(1,6): #for machine pred
            sub=pred[(4*(i-1)):(4*i)]
            aux=aux*np.asarray(sub)
            
        aux=aux/np.sum(aux)
        final.append(np.argmax(aux))
        confianzas.append(aux[np.argmax(aux)])  
           
    predichos_new=[]
    for i in range(len(temp)):
        predichos_new.append(final[i])
    return predichos_new,final,confianzas
    
def predecir_modelos_average(bs,predichos_all): #average
    temp=predichos_all.copy()
    final=[]
    confianzas=[]
    for pred in temp:
        aux=np.zeros(4)
        for i in range(1,6):
            sub=pred[(4*(i-1)):(4*i)]
            aux=aux+np.asarray(sub)
            
        aux=aux/5.0 #dividido en el total de machines
        final.append(np.argmax(aux))
        confianzas.append(aux[np.argmax(aux)])  
           
    predichos_new=[]
    for i in range(len(temp)):
        predichos_new.append(final[i])
    return predichos_new,final,confianzas


## comité 0.15

In [6]:
prob=0.15
path='../../../new_data_augmented/harassment_baselines/'
bs=32
    
fs_macro_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

fs_macro_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

best={'acc':[], 'none':[], 'macro':[]}
norm={'acc':[], 'none':[], 'macro':[]}
voting={'acc':[], 'none':[], 'macro':[]}
best_test={'acc':[], 'none':[], 'macro':[]}
norm_test={'acc':[], 'none':[], 'macro':[]}
voting_test={'acc':[], 'none':[], 'macro':[]}

for j in range(1,6):
    print ("")
    print ("---------------- EJECUCIÓN #"+str(j)+'----------------')
    cnn1= load_model(path+str(prob)+'/cnn1_'+str(j)+'-exec_app1_Top_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()})  
    cnn2= load_model(path+str(prob)+'/cnn2_'+str(j)+'-exec_app1_Top_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn1= load_model(path+str(prob)+'/rnn1_'+str(j)+'-exec_app1_Top_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn2= load_model(path+str(prob)+'/rnn2_'+str(j)+'-exec_app1_Top_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn3= load_model(path+str(prob)+'/rnn3_'+str(j)+'-exec_app1_Top_1.h5') 
    
    list_models=['cnn1', 'cnn2', 'rnn1', 'rnn2', 'rnn3']
    index_models=np.arange(5)
    dict_models=dict((key, value) for (key, value) in zip(index_models,list_models))
    modelos=[cnn1, cnn2, rnn1, rnn2, rnn3]
    ind=np.arange(5)
    dict_trainedModel=dict((key, value) for (key, value) in zip(ind,modelos))
    
    print ("Agregando predicciones Val set")
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_val, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_v, predicho)  
        f1_ma=f1_score(etiq_v, predicho, average='macro')
        f1_no=f1_score(etiq_v, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_v, predicho))
        fs_macro_val[dict_models[i]].append(f1_ma)
        fs_none_val[dict_models[i]].append(f1_no)
        accs_val[dict_models[i]].append(acc)
        confusions_val[dict_models[i]].append(matriz)
        
    predicciones_all_val=[]
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_val.append(dict_trainedModel[i].predict(x_val, batch_size=bs))
    predichos_all_val=np.concatenate(np.asarray(predicciones_all_val),axis=-1)
    
    print ("Agregando predicciones Test set", dict_models[i])    
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_test, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_t, predicho)  
        f1_ma=f1_score(etiq_t, predicho, average='macro')
        f1_no=f1_score(etiq_t, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_t, predicho))
        fs_macro_test[dict_models[i]].append(f1_ma)
        fs_none_test[dict_models[i]].append(f1_no)
        accs_test[dict_models[i]].append(acc)
        confusions_test[dict_models[i]].append(matriz)

    predicciones_all_test=[]
    bs=32
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_test.append(dict_trainedModel[i].predict(x_test, batch_size=bs))
    predichos_all_test=np.concatenate(np.asarray(predicciones_all_test),axis=-1)
    
    print ("--------VALIDATION SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score NIPS:",f1)
    best['acc'].append(acc_comite)
    best['macro'].append(f1)
    best['none'].append(f1_no)
    
    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None) 
    print("F1-score NIPS:",f1)
    norm['acc'].append(acc_comite)
    norm['macro'].append(f1)
    norm['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score NIPS:",f1)
    voting['acc'].append(acc_comite)
    voting['macro'].append(f1)
    voting['none'].append(f1_no)
    
    print ("--------TESTING SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score NIPS:",f1)
    best_test['acc'].append(acc_comite)
    best_test['macro'].append(f1)
    best_test['none'].append(f1_no)

    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None) 
    print("F1-score NIPS:",f1)
    norm_test['acc'].append(acc_comite)
    norm_test['macro'].append(f1)
    norm_test['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score NIPS:",f1)
    voting_test['acc'].append(acc_comite)
    voting_test['macro'].append(f1)
    voting_test['none'].append(f1_no)



---------------- EJECUCIÓN #1----------------





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best Fit
Accuracy: 0.72
F1-macro: 0.4076334978030106
F1-score NIPS: 0.4076334978030106

Committee Norm
Accuracy: 0.7496470588235294
F1-macro: 0.4080264873743366
F1-score NIPS: 0.4080264873743366

Committee Voting
Accuracy: 0.7656470588235295
F1-macro: 0.4158248439689737
F1-score NIPS: 0.4158248439689737
--------TESTING SET--------

Committee Best Fit
Accuracy: 0.7305699481865285
F1-macro: 0.38462668541817346
F1-score NIPS: 0.38462668541817346

Committee Norm
Accuracy: 0.7602449364107395
F1-macro: 0.38135547880811915
F1-score NIPS: 0.38135547880811915

Committee Voting
Accuracy: 0.7828544512482336
F1-macro: 0.40063719071

In [7]:
norm_test['none']

[array([0.85677993, 0.01785714, 0.07142857, 0.57935627]),
 array([0.82492381, 0.17045455, 0.24460432, 0.57464213]),
 array([0.78230337, 0.03478261, 0.075     , 0.50207814]),
 array([0.75944767, 0.        , 0.07228916, 0.48233487]),
 array([0.85796178, 0.        , 0.07317073, 0.59234973])]

In [8]:
m=5.0

print ("[TEST] Acc Best comité:",np.sum(np.asarray(best_test['acc'])/m))
temp=np.zeros(4)
for result in best_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Best comité:" ,temp/m)
print ("[TEST] F1 macro Best comité:",np.sum(np.asarray(best_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Norm comité:",np.sum(np.asarray(norm_test['acc'])/m))
temp=np.zeros(4)
for result in norm_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Norm comité:" ,temp/m)
print ("[TEST] F1 macro Norm comité:",np.sum(np.asarray(norm_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Voting comité:",np.sum(np.asarray(voting_test['acc'])/m))
temp=np.zeros(4)
for result in voting_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Voting comité:" ,temp/m)
print ("[TEST] F1 macro Voting comité:",np.sum(np.asarray(voting_test['macro'])/m))


[TEST] Acc Best comité: 0.6243994347621291
[TEST] F1-score NIPS Best comité: [0.73596574 0.09067093 0.13687677 0.52093618]
[TEST] F1 macro Best comité: 0.3711124054912137
-----------
[TEST] Acc Norm comité: 0.7111634479510127
[TEST] F1-score NIPS Norm comité: [0.81628331 0.04461886 0.10729856 0.54615223]
[TEST] F1 macro Norm comité: 0.378588238509594
-----------
[TEST] Acc Voting comité: 0.7268016957136129
[TEST] F1-score NIPS Voting comité: [0.82804081 0.06288643 0.12937531 0.57018333]
[TEST] F1 macro Voting comité: 0.39762147127720504


## Comité 0.5

In [9]:
prob=0.5
path='../../../new_data_augmented/harassment_baselines/'
bs=32
    
fs_macro_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

fs_macro_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

best={'acc':[], 'none':[], 'macro':[]}
norm={'acc':[], 'none':[], 'macro':[]}
voting={'acc':[], 'none':[], 'macro':[]}
best_test={'acc':[], 'none':[], 'macro':[]}
norm_test={'acc':[], 'none':[], 'macro':[]}
voting_test={'acc':[], 'none':[], 'macro':[]}

for j in range(1,6):
    print ("")
    print ("---------------- EJECUCIÓN #"+str(j)+'----------------')
    cnn1= load_model(path+str(prob)+'/cnn1_'+str(j)+'-exec_app1_Top_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()})  
    cnn2= load_model(path+str(prob)+'/cnn2_'+str(j)+'-exec_app1_Top_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn1= load_model(path+str(prob)+'/rnn1_'+str(j)+'-exec_app1_Top_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn2= load_model(path+str(prob)+'/rnn2_'+str(j)+'-exec_app1_Top_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn3= load_model(path+str(prob)+'/rnn3_'+str(j)+'-exec_app1_Top_1.h5') 
    
    list_models=['cnn1', 'cnn2', 'rnn1', 'rnn2', 'rnn3']
    index_models=np.arange(5)
    dict_models=dict((key, value) for (key, value) in zip(index_models,list_models))
    modelos=[cnn1, cnn2, rnn1, rnn2, rnn3]
    ind=np.arange(5)
    dict_trainedModel=dict((key, value) for (key, value) in zip(ind,modelos))
    
    print ("Agregando predicciones Val set")
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_val, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_v, predicho)  
        f1_ma=f1_score(etiq_v, predicho, average='macro')
        f1_no=f1_score(etiq_v, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_v, predicho))
        fs_macro_val[dict_models[i]].append(f1_ma)
        fs_none_val[dict_models[i]].append(f1_no)
        accs_val[dict_models[i]].append(acc)
        confusions_val[dict_models[i]].append(matriz)
        
    predicciones_all_val=[]
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_val.append(dict_trainedModel[i].predict(x_val, batch_size=bs))
    predichos_all_val=np.concatenate(np.asarray(predicciones_all_val),axis=-1)
    
    print ("Agregando predicciones Test set", dict_models[i])    
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_test, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_t, predicho)  
        f1_ma=f1_score(etiq_t, predicho, average='macro')
        f1_no=f1_score(etiq_t, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_t, predicho))
        fs_macro_test[dict_models[i]].append(f1_ma)
        fs_none_test[dict_models[i]].append(f1_no)
        accs_test[dict_models[i]].append(acc)
        confusions_test[dict_models[i]].append(matriz)

    predicciones_all_test=[]
    bs=32
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_test.append(dict_trainedModel[i].predict(x_test, batch_size=bs))
    predichos_all_test=np.concatenate(np.asarray(predicciones_all_test),axis=-1)
    
    print ("--------VALIDATION SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score NIPS:",f1)
    best['acc'].append(acc_comite)
    best['macro'].append(f1)
    best['none'].append(f1_no)
    
    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None) 
    print("F1-score NIPS:",f1)
    norm['acc'].append(acc_comite)
    norm['macro'].append(f1)
    norm['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score NIPS:",f1)
    voting['acc'].append(acc_comite)
    voting['macro'].append(f1)
    voting['none'].append(f1_no)
    
    print ("--------TESTING SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score NIPS:",f1)
    best_test['acc'].append(acc_comite)
    best_test['macro'].append(f1)
    best_test['none'].append(f1_no)

    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None) 
    print("F1-score NIPS:",f1)
    norm_test['acc'].append(acc_comite)
    norm_test['macro'].append(f1)
    norm_test['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score NIPS:",f1)
    voting_test['acc'].append(acc_comite)
    voting_test['macro'].append(f1)
    voting_test['none'].append(f1_no)



---------------- EJECUCIÓN #1----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best Fit
Accuracy: 0.6263529411764706
F1-macro: 0.35558903467113223
F1-score NIPS: 0.35558903467113223

Committee Norm
Accuracy: 0.6461176470588236
F1-macro: 0.3591800017320015
F1-score NIPS: 0.3591800017320015

Committee Voting
Accuracy: 0.664
F1-macro: 0.36870682133874527
F1-score NIPS: 0.36870682133874527
--------TESTING SET--------

Committee Best Fit
Accuracy: 0.6354215732454075
F1-macro: 0.3408691812916822
F1-score NIPS: 0.3408691812916822

Committee Norm
Accuracy: 0.6636834667922751
F1-macro: 0.3445191868099694
F1-score NIPS: 0.3445191868099694

Committee Voting
Accuracy: 0.6778144135657089
F1-macro: 0.35697646050673343
F1-score NIPS: 0.35697646050673343

---------------- EJECUCIÓN #2----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best Fit
Accurac

In [10]:
norm_test['none']

[array([0.7801618 , 0.03571429, 0.075     , 0.48720066]),
 array([0.73084043, 0.03636364, 0.07058824, 0.44888889]),
 array([0.70168712, 0.05042017, 0.15384615, 0.44507846]),
 array([0.74908291, 0.01785714, 0.20408163, 0.48057098]),
 array([0.84842584, 0.05263158, 0.05128205, 0.58787256])]

In [11]:
m=5.0

print ("[TEST] Acc Best comité:",np.sum(np.asarray(best_test['acc'])/m))
temp=np.zeros(4)
for result in best_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Best comité:" ,temp/m)
print ("[TEST] F1 macro Best comité:",np.sum(np.asarray(best_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Norm comité:",np.sum(np.asarray(norm_test['acc'])/m))
temp=np.zeros(4)
for result in norm_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Norm comité:" ,temp/m)
print ("[TEST] F1 macro Norm comité:",np.sum(np.asarray(norm_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Voting comité:",np.sum(np.asarray(voting_test['acc'])/m))
temp=np.zeros(4)
for result in voting_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Voting comité:" ,temp/m)
print ("[TEST] F1 macro Voting comité:",np.sum(np.asarray(voting_test['macro'])/m))


[TEST] Acc Best comité: 0.6059349976448423
[TEST] F1-score NIPS Best comité: [0.72279813 0.1085847  0.12764349 0.46696791]
[TEST] F1 macro Best comité: 0.356498558997753
-----------
[TEST] Acc Norm comité: 0.6482336316533208
[TEST] F1-score NIPS Norm comité: [0.76203962 0.03859736 0.11095961 0.48992231]
[TEST] F1 macro Norm comité: 0.35037972582268306
-----------
[TEST] Acc Voting comité: 0.6693358455016485
[TEST] F1-score NIPS Voting comité: [0.7810994  0.04544463 0.11578284 0.51014871]
[TEST] F1 macro Voting comité: 0.3631188931940822


## Comité 0.85

In [12]:
prob=0.85
path='../../../new_data_augmented/harassment_baselines/'
bs=32
    
fs_macro_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

fs_macro_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

best={'acc':[], 'none':[], 'macro':[]}
norm={'acc':[], 'none':[], 'macro':[]}
voting={'acc':[], 'none':[], 'macro':[]}
best_test={'acc':[], 'none':[], 'macro':[]}
norm_test={'acc':[], 'none':[], 'macro':[]}
voting_test={'acc':[], 'none':[], 'macro':[]}

for j in range(1,6):
    print ("")
    print ("---------------- EJECUCIÓN #"+str(j)+'----------------')
    cnn1= load_model(path+str(prob)+'/cnn1_'+str(j)+'-exec_app1_Top_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()})  
    cnn2= load_model(path+str(prob)+'/cnn2_'+str(j)+'-exec_app1_Top_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn1= load_model(path+str(prob)+'/rnn1_'+str(j)+'-exec_app1_Top_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn2= load_model(path+str(prob)+'/rnn2_'+str(j)+'-exec_app1_Top_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn3= load_model(path+str(prob)+'/rnn3_'+str(j)+'-exec_app1_Top_1.h5') 
    
    list_models=['cnn1', 'cnn2', 'rnn1', 'rnn2', 'rnn3']
    index_models=np.arange(5)
    dict_models=dict((key, value) for (key, value) in zip(index_models,list_models))
    modelos=[cnn1, cnn2, rnn1, rnn2, rnn3]
    ind=np.arange(5)
    dict_trainedModel=dict((key, value) for (key, value) in zip(ind,modelos))
    
    print ("Agregando predicciones Val set")
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_val, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_v, predicho)  
        f1_ma=f1_score(etiq_v, predicho, average='macro')
        f1_no=f1_score(etiq_v, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_v, predicho))
        fs_macro_val[dict_models[i]].append(f1_ma)
        fs_none_val[dict_models[i]].append(f1_no)
        accs_val[dict_models[i]].append(acc)
        confusions_val[dict_models[i]].append(matriz)
        
    predicciones_all_val=[]
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_val.append(dict_trainedModel[i].predict(x_val, batch_size=bs))
    predichos_all_val=np.concatenate(np.asarray(predicciones_all_val),axis=-1)
    
    print ("Agregando predicciones Test set", dict_models[i])    
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_test, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_t, predicho)  
        f1_ma=f1_score(etiq_t, predicho, average='macro')
        f1_no=f1_score(etiq_t, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_t, predicho))
        fs_macro_test[dict_models[i]].append(f1_ma)
        fs_none_test[dict_models[i]].append(f1_no)
        accs_test[dict_models[i]].append(acc)
        confusions_test[dict_models[i]].append(matriz)

    predicciones_all_test=[]
    bs=32
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_test.append(dict_trainedModel[i].predict(x_test, batch_size=bs))
    predichos_all_test=np.concatenate(np.asarray(predicciones_all_test),axis=-1)
    
    print ("--------VALIDATION SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score NIPS:",f1)
    best['acc'].append(acc_comite)
    best['macro'].append(f1)
    best['none'].append(f1_no)
    
    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None) 
    print("F1-score NIPS:",f1)
    norm['acc'].append(acc_comite)
    norm['macro'].append(f1)
    norm['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score NIPS:",f1)
    voting['acc'].append(acc_comite)
    voting['macro'].append(f1)
    voting['none'].append(f1_no)
    
    print ("--------TESTING SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score NIPS:",f1)
    best_test['acc'].append(acc_comite)
    best_test['macro'].append(f1)
    best_test['none'].append(f1_no)

    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None) 
    print("F1-score NIPS:",f1)
    norm_test['acc'].append(acc_comite)
    norm_test['macro'].append(f1)
    norm_test['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score NIPS:",f1)
    voting_test['acc'].append(acc_comite)
    voting_test['macro'].append(f1)
    voting_test['none'].append(f1_no)



---------------- EJECUCIÓN #1----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best Fit
Accuracy: 0.5929411764705882
F1-macro: 0.3494094395873327
F1-score NIPS: 0.3494094395873327

Committee Norm
Accuracy: 0.6531764705882352
F1-macro: 0.36700349791029374
F1-score NIPS: 0.36700349791029374

Committee Voting
Accuracy: 0.6418823529411765
F1-macro: 0.35921400459675445
F1-score NIPS: 0.35921400459675445
--------TESTING SET--------

Committee Best Fit
Accuracy: 0.599152143193594
F1-macro: 0.38156242250796707
F1-score NIPS: 0.38156242250796707

Committee Norm
Accuracy: 0.6556759302873293
F1-macro: 0.39188356872138796
F1-score NIPS: 0.39188356872138796

Committee Voting
Accuracy: 0.6561469618464437
F1-macro: 0.4086192161927772
F1-score NIPS: 0.4086192161927772

---------------- EJECUCIÓN #2----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Be

In [13]:
m=5.0

print ("[TEST] Acc Best comité:",np.sum(np.asarray(best_test['acc'])/m))
temp=np.zeros(4)
for result in best_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Best comité:" ,temp/m)
print ("[TEST] F1 macro Best comité:",np.sum(np.asarray(best_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Norm comité:",np.sum(np.asarray(norm_test['acc'])/m))
temp=np.zeros(4)
for result in norm_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Norm comité:" ,temp/m)
print ("[TEST] F1 macro Norm comité:",np.sum(np.asarray(norm_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Voting comité:",np.sum(np.asarray(voting_test['acc'])/m))
temp=np.zeros(4)
for result in voting_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Voting comité:" ,temp/m)
print ("[TEST] F1 macro Voting comité:",np.sum(np.asarray(voting_test['macro'])/m))


[TEST] Acc Best comité: 0.5995289684408855
[TEST] F1-score NIPS Best comité: [0.72140282 0.14232224 0.13042259 0.47312257]
[TEST] F1 macro Best comité: 0.366817555842783
-----------
[TEST] Acc Norm comité: 0.6508714083843617
[TEST] F1-score NIPS Norm comité: [0.76414593 0.0938708  0.11005172 0.49319122]
[TEST] F1 macro Norm comité: 0.3653149163545282
-----------
[TEST] Acc Voting comité: 0.6423928403203014
[TEST] F1-score NIPS Voting comité: [0.75778689 0.09659616 0.10188824 0.48998068]
[TEST] F1 macro Voting comité: 0.36156299208843323


## Comité 0.0 sin pesos

In [6]:
prob=0.0
path='../../../new_data_augmented/harassment_baselines/'
bs=32
    
fs_macro_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

fs_macro_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

best={'acc':[], 'none':[], 'macro':[]}
norm={'acc':[], 'none':[], 'macro':[]}
voting={'acc':[], 'none':[], 'macro':[]}
best_test={'acc':[], 'none':[], 'macro':[]}
norm_test={'acc':[], 'none':[], 'macro':[]}
voting_test={'acc':[], 'none':[], 'macro':[]}

for j in range(1,6):
    print ("")
    print ("---------------- EJECUCIÓN #"+str(j)+'----------------')
    cnn1= load_model(path+str(prob)+'/cnn1_'+str(j)+'-exec.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()})  
    cnn2= load_model(path+str(prob)+'/cnn2_'+str(j)+'-exec.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn1= load_model(path+str(prob)+'/rnn1_'+str(j)+'-exec.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn2= load_model(path+str(prob)+'/rnn2_'+str(j)+'-exec.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn3= load_model(path+str(prob)+'/rnn3_'+str(j)+'-exec.h5') 
    
    list_models=['cnn1', 'cnn2', 'rnn1', 'rnn2', 'rnn3']
    index_models=np.arange(5)
    dict_models=dict((key, value) for (key, value) in zip(index_models,list_models))
    modelos=[cnn1, cnn2, rnn1, rnn2, rnn3]
    ind=np.arange(5)
    dict_trainedModel=dict((key, value) for (key, value) in zip(ind,modelos))
    
    print ("Agregando predicciones Val set")
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_val, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_v, predicho)  
        f1_ma=f1_score(etiq_v, predicho, average='macro')
        f1_no=f1_score(etiq_v, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_v, predicho))
        fs_macro_val[dict_models[i]].append(f1_ma)
        fs_none_val[dict_models[i]].append(f1_no)
        accs_val[dict_models[i]].append(acc)
        confusions_val[dict_models[i]].append(matriz)
        
    predicciones_all_val=[]
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_val.append(dict_trainedModel[i].predict(x_val, batch_size=bs))
    predichos_all_val=np.concatenate(np.asarray(predicciones_all_val),axis=-1)
    
    print ("Agregando predicciones Test set", dict_models[i])    
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_test, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_t, predicho)  
        f1_ma=f1_score(etiq_t, predicho, average='macro')
        f1_no=f1_score(etiq_t, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_t, predicho))
        fs_macro_test[dict_models[i]].append(f1_ma)
        fs_none_test[dict_models[i]].append(f1_no)
        accs_test[dict_models[i]].append(acc)
        confusions_test[dict_models[i]].append(matriz)

    predicciones_all_test=[]
    bs=32
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_test.append(dict_trainedModel[i].predict(x_test, batch_size=bs))
    predichos_all_test=np.concatenate(np.asarray(predicciones_all_test),axis=-1)
    
    print ("--------VALIDATION SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score NIPS:",f1)
    best['acc'].append(acc_comite)
    best['macro'].append(f1)
    best['none'].append(f1_no)
    
    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None) 
    print("F1-score NIPS:",f1)
    norm['acc'].append(acc_comite)
    norm['macro'].append(f1)
    norm['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score NIPS:",f1)
    voting['acc'].append(acc_comite)
    voting['macro'].append(f1)
    voting['none'].append(f1_no)
    
    print ("--------TESTING SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score NIPS:",f1)
    best_test['acc'].append(acc_comite)
    best_test['macro'].append(f1)
    best_test['none'].append(f1_no)

    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None) 
    print("F1-score NIPS:",f1)
    norm_test['acc'].append(acc_comite)
    norm_test['macro'].append(f1)
    norm_test['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score NIPS:",f1)
    voting_test['acc'].append(acc_comite)
    voting_test['macro'].append(f1)
    voting_test['none'].append(f1_no)



---------------- EJECUCIÓN #1----------------





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best Fit
Accuracy: 0.8687058823529412
F1-macro: 0.4718352098418117
F1-score NIPS: 0.4718352098418117

Committee Norm
Accuracy: 0.88
F1-macro: 0.4511999062032155
F1-score NIPS: 0.4511999062032155

Committee Voting
Accuracy: 0.8809411764705882
F1-macro: 0.4571346468879125
F1-score NIPS: 0.4571346468879125
--------TESTING SET--------

Committee Best Fit
Accuracy: 0.8276024493641074
F1-macro: 0.40620110889633043
F1-score NIPS: 0.40620110889633043

Committee Norm
Accuracy: 0.83843617522374
F1-macro: 0.41032466536378176
F1-score NIPS: 0.41032466536378176

Committee Voting
Accuracy: 0.8379651436646255
F1-macro: 0.4154224956986

In [7]:
m=5.0

print ("[TEST] Acc Best comité:",np.sum(np.asarray(best_test['acc'])/m))
temp=np.zeros(4)
for result in best_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Best comité:" ,temp/m)
print ("[TEST] F1 macro Best comité:",np.sum(np.asarray(best_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Norm comité:",np.sum(np.asarray(norm_test['acc'])/m))
temp=np.zeros(4)
for result in norm_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Norm comité:" ,temp/m)
print ("[TEST] F1 macro Norm comité:",np.sum(np.asarray(norm_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Voting comité:",np.sum(np.asarray(voting_test['acc'])/m))
temp=np.zeros(4)
for result in voting_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Voting comité:" ,temp/m)
print ("[TEST] F1 macro Voting comité:",np.sum(np.asarray(voting_test['macro'])/m))

[TEST] Acc Best comité: 0.8274140367404615
[TEST] F1-score NIPS Best comité: [0.90366597 0.03130252 0.04765457 0.6827535 ]
[TEST] F1 macro Best comité: 0.4163441379097411
-----------
[TEST] Acc Norm comité: 0.839284032030146
[TEST] F1-score NIPS Norm comité: [0.9080034  0.00373832 0.0147561  0.71847359]
[TEST] F1 macro Norm comité: 0.4112428518277096
-----------
[TEST] Acc Voting comité: 0.8430522845030619
[TEST] F1-score NIPS Voting comité: [0.91027774 0.         0.01855501 0.72878386]
[TEST] F1 macro Voting comité: 0.41440415134950387


# Comité con pesos

In [8]:
prob=0.0
path='../../../new_data_augmented/harassment_baselines/'
bs=32
    
fs_macro_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

fs_macro_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

best={'acc':[], 'none':[], 'macro':[]}
norm={'acc':[], 'none':[], 'macro':[]}
voting={'acc':[], 'none':[], 'macro':[]}
best_test={'acc':[], 'none':[], 'macro':[]}
norm_test={'acc':[], 'none':[], 'macro':[]}
voting_test={'acc':[], 'none':[], 'macro':[]}

for j in range(1,6):
    print ("")
    print ("---------------- EJECUCIÓN #"+str(j)+'----------------')
    cnn1= load_model(path+str(prob)+'_cw/cnn1_'+str(j)+'-exec.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()})  
    cnn2= load_model(path+str(prob)+'_cw/cnn2_'+str(j)+'-exec.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn1= load_model(path+str(prob)+'_cw/rnn1_'+str(j)+'-exec.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn2= load_model(path+str(prob)+'_cw/rnn2_'+str(j)+'-exec.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn3= load_model(path+str(prob)+'_cw/rnn3_'+str(j)+'-exec.h5') 
    
    list_models=['cnn1', 'cnn2', 'rnn1', 'rnn2', 'rnn3']
    index_models=np.arange(5)
    dict_models=dict((key, value) for (key, value) in zip(index_models,list_models))
    modelos=[cnn1, cnn2, rnn1, rnn2, rnn3]
    ind=np.arange(5)
    dict_trainedModel=dict((key, value) for (key, value) in zip(ind,modelos))
    
    print ("Agregando predicciones Val set")
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_val, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_v, predicho)  
        f1_ma=f1_score(etiq_v, predicho, average='macro')
        f1_no=f1_score(etiq_v, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_v, predicho))
        fs_macro_val[dict_models[i]].append(f1_ma)
        fs_none_val[dict_models[i]].append(f1_no)
        accs_val[dict_models[i]].append(acc)
        confusions_val[dict_models[i]].append(matriz)
        
    predicciones_all_val=[]
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_val.append(dict_trainedModel[i].predict(x_val, batch_size=bs))
    predichos_all_val=np.concatenate(np.asarray(predicciones_all_val),axis=-1)
    
    print ("Agregando predicciones Test set", dict_models[i])    
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_test, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_t, predicho)  
        f1_ma=f1_score(etiq_t, predicho, average='macro')
        f1_no=f1_score(etiq_t, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_t, predicho))
        fs_macro_test[dict_models[i]].append(f1_ma)
        fs_none_test[dict_models[i]].append(f1_no)
        accs_test[dict_models[i]].append(acc)
        confusions_test[dict_models[i]].append(matriz)

    predicciones_all_test=[]
    bs=32
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_test.append(dict_trainedModel[i].predict(x_test, batch_size=bs))
    predichos_all_test=np.concatenate(np.asarray(predicciones_all_test),axis=-1)
    
    print ("--------VALIDATION SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score NIPS:",f1)
    best['acc'].append(acc_comite)
    best['macro'].append(f1)
    best['none'].append(f1_no)
    
    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None) 
    print("F1-score NIPS:",f1)
    norm['acc'].append(acc_comite)
    norm['macro'].append(f1)
    norm['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score NIPS:",f1)
    voting['acc'].append(acc_comite)
    voting['macro'].append(f1)
    voting['none'].append(f1_no)
    
    print ("--------TESTING SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score NIPS:",f1)
    best_test['acc'].append(acc_comite)
    best_test['macro'].append(f1)
    best_test['none'].append(f1_no)

    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None) 
    print("F1-score NIPS:",f1)
    norm_test['acc'].append(acc_comite)
    norm_test['macro'].append(f1)
    norm_test['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score NIPS:",f1)
    voting_test['acc'].append(acc_comite)
    voting_test['macro'].append(f1)
    voting_test['none'].append(f1_no)



---------------- EJECUCIÓN #1----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best Fit
Accuracy: 0.848
F1-macro: 0.45351107432201354
F1-score NIPS: 0.45351107432201354

Committee Norm
Accuracy: 0.88
F1-macro: 0.4662639744174725
F1-score NIPS: 0.4662639744174725

Committee Voting
Accuracy: 0.8875294117647059
F1-macro: 0.464273563894934
F1-score NIPS: 0.464273563894934
--------TESTING SET--------

Committee Best Fit
Accuracy: 0.8064060292039567
F1-macro: 0.4415204469832272
F1-score NIPS: 0.4415204469832272

Committee Norm
Accuracy: 0.83843617522374
F1-macro: 0.4407209158700522
F1-score NIPS: 0.4407209158700522

Committee Voting
Accuracy: 0.844559585492228
F1-macro: 0.44734904495635874
F1-score NIPS: 0.44734904495635874

---------------- EJECUCIÓN #2----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best Fit
Accuracy: 0.8536470588235294

In [9]:
m=5.0

print ("[TEST] Acc Best comité:",np.sum(np.asarray(best_test['acc'])/m))
temp=np.zeros(4)
for result in best_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Best comité:" ,temp/m)
print ("[TEST] F1 macro Best comité:",np.sum(np.asarray(best_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Norm comité:",np.sum(np.asarray(norm_test['acc'])/m))
temp=np.zeros(4)
for result in norm_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Norm comité:" ,temp/m)
print ("[TEST] F1 macro Norm comité:",np.sum(np.asarray(norm_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Voting comité:",np.sum(np.asarray(voting_test['acc'])/m))
temp=np.zeros(4)
for result in voting_test['none']:
    temp+=result
print ("[TEST] F1-score NIPS Voting comité:" ,temp/m)
print ("[TEST] F1 macro Voting comité:",np.sum(np.asarray(voting_test['macro'])/m))

[TEST] Acc Best comité: 0.8067828544512483
[TEST] F1-score NIPS Best comité: [0.89226876 0.06325765 0.06101025 0.65929362]
[TEST] F1 macro Best comité: 0.4189575707011054
-----------
[TEST] Acc Norm comité: 0.8316533207724918
[TEST] F1-score NIPS Norm comité: [0.90438829 0.01461461 0.03918182 0.68406926]
[TEST] F1 macro Norm comité: 0.41056349693553595
-----------
[TEST] Acc Voting comité: 0.8317475270843147
[TEST] F1-score NIPS Voting comité: [0.90476948 0.0425335  0.05039476 0.68995598]
[TEST] F1 macro Voting comité: 0.4219134296330304
